<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2024/images/logo.png?raw=true" alt="2023年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

大規模なゲノムデータを用いた解析: Genomic Prediction その2
---

　今回は、GenomicPredictionの第２回です。

大規模ゲノムデータ解析シリーズ:
- GWAS: Genome wide association study(GWAS)
- **Genomic Prediction入門 : Genomic Predictionとは**, 機械学習の基本, 統計モデル, 育種への応用...等々


# 今回の勉強内容
　このテキストでは、Genomic Prediction入門の第2回として、以下を勉強します。

　前回の重回帰式で構築した予測モデルには問題があるので、その問題を避けてより良いモデルの構築を目指します。

1. 前回構築したモデルの問題点
  1. 他の集団に対しても精度が確保できているのか？
  1. SNPの効果がおかしい
1. 過学習
  1. 過学習とは？
1. 問題点を改善するために
  1. 学習データとテストデータへの分割
  1. 変数の数を減らす

---

　一度次のコードセルを実行して、実習に必要なライブラリのインストールや、プログラム、サンプルファイルのダウンロードしてください。

In [ ]:
################################
##  以下の実習の前に一度実行してください。##
################################

## プログラム・サンプルファイルのダウンロード
!wget -q -O 'GP_sample_genotype.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/GP_sample_genotype.csv
!wget -q -O 'GP_sample_phenotype.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/GP_sample_phenotype.csv
!wget -q -O genomic_prediction.py https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/scripts/genomic_prediction_1.py?raw=true
## 確認
!ls | grep  -e 'GP_sample_genotype.csv' -e 'GP_sample_phenotype.csv' -e 'genomic_prediction.py'

# 1. 前回構築したモデルの問題点

前回は重回帰モデルを使ってNAM集団で予測モデルを構築してみました。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_data.png?raw=true" alt="GP_data" height="400px" align="middle">

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_sample_model3.png?raw=true" alt="GP_sample_model3" height="80px">

しかし、構築したモデルにはいくつかの問題点がありそうです。

まずはその問題を見ていきましょう。

In [ ]:
# ライブラリの読み込み
import pandas as pd
import numpy as np

# データセットの読み込み
# Genotype data
genotype = pd.read_csv("GP_sample_genotype.csv")
# Phenotype data
phenotype = pd.read_csv("GP_sample_phenotype.csv")

In [ ]:
# ライブラリの読み込み
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.linear_model import LinearRegression

# 線形回帰式
model = LinearRegression()
model = model.fit(genotype.iloc[:, 2:].T, phenotype["GN_mean"])
equation = "y = "
for i, j in enumerate(model.coef_):
    equation += "SNP{}×{} + ".format(str(i+1), str(j))
equation += str(model.intercept_)
print(equation)

# 予測精度
y_preds = model.predict(genotype.iloc[:, 2:].T)
plt.scatter(phenotype["GN_mean"], y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(phenotype["GN_mean"], y_preds)[0, 1]))

## 1-1. 正しい予測精度が分からない

今回はデータの遺伝子型と形質値の関係性を説明する様な回帰モデルを作っています。

与えられたデータを上手く説明するように回帰式は作られているので、予測値と実測値が近いのは当然です。

一見、今回の予測モデルは非常に高い予測精度を持っている様に見えますが、これは**モデルがデータに上手く当てはまっている**ことを示しているだけで、モデルの汎用性が高いかどうか(つまり、他の集団に対しても予測モデルは高い精度を示すかどうか)は示していません。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_other_pop.png?raw=true" alt="GP_other_pop" height="400px">

そのため、予測モデルを作成するときには、新たにデータが手に入った時に、そのデータに対しても高い予測精度を示せるのかどうか、を考える必要があります。

今回はその視点が全く無い状態でした。

## 1-2. SNPの効果がおかしい

今回は、回帰式として予測モデルを構築したので、各SNPの遺伝子型が変わった時の効果の大きさが分かります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_sample_model3.png?raw=true" alt="GP_sample_model3" height="80px">

この式で言うと$\beta_1$~$\beta_{6366}$の値です。

この$\beta$の値が大きいと、その場所のSNPの遺伝子型が変わった時に$y$(形質値)に大きな影響を及ぼすことになります。

下記のコードで、今回作った回帰式のそれぞれのSNPの効果を可視化してみましょう。

In [ ]:
plt.figure(figsize=(12,3))
plt.plot(model.coef_)
plt.xlabel("SNP")
plt.ylabel("coefficient (SNP effect)")
plt.show()

まず、最初の方のSNPに極端に効果が大きい(小さい)SNPがあることが分かると思います。

極端にプラスの効果のすぐ近くにマイナスの効果のSNPがあるのは変な感じがしますね。

In [ ]:
print(equation)

更に、今回 $y$ (形質値)としては、籾数を扱っていました。

そのため、$y$ のスケールとしては数十粒~数百粒の大きさになります。

しかし、それぞれのSNPの遺伝子型が変わった時の効果を見てみると`+29004183747.357655`とか`-18302901843.52791`とか、異様にスケールが大きくなっています。

例えば１つ目のSNPの遺伝子型がひとめぼれ型から変異すると、籾数が290億粒増える計算になります…が、当然SNPの効果がそんなに大きいはずは無いです。

それに今回は全てのSNPが何らかの効果を持っている式になっていますが、全ての遺伝子が形質に影響を与えている訳では無いので、全領域のSNPが効果を持っているのはおかしいです。

その辺りを鑑みると、どうもこの回帰式は現実的におかしな式になってしまっているようです。

なぜこんなことが起きてしまうのかというと、それは**多重共線性や過学習**が起きているからです。

## (参考)1-3. 多重共線性の問題

まず一つ目の問題の原因として、多重共線性が挙げられます。

重回帰分析では、説明変数の中に相関係数が高い組み合わせが存在すると、回帰係数の推定値が不規則に変化してしまう可能性が高いです。

この様なケースを多重共線性と呼びます(マルチコと呼んだりもする)。

特にSNPの情報を使用する場合、隣同士のSNPは連鎖の関係から、集団の遺伝子型が似たような状況になる場合が多く、そのまま入力データとして重回帰分析に用いると多重共線性となることが多いです。

多重共線性を回避するためには、相関の強い説明変数を減らしたり、PCA(主成分分析)等を使用し複数の変数をまとめてしまう等、様々なアプローチがあります。

# 2. 過学習(overfitting)

次に問題の原因として過学習があります。

過学習というのはどういうものかというと、学習したデータ(モデルを作るために使用したデータ)に対し、過剰に適合してしまった結果、別のデータや新しいデータに対して予測精度が低くなってしまうことを言います。

例えば下の様なデータがあったとします。

In [ ]:
obs_x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
obs_y = [101, 105, 103, 106, 108, 110, 107, 108, 114, 110]
plt.scatter(obs_x, obs_y)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

人間の直感的には、xが大きくなるにつれ、yが大きくなる関係なんだろうなという気がしますね。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_overfit_1.png?raw=true" alt="GP_overfit_1" height="250px">

下のコードで、この直感を回帰式で表すことが出来ます。

ではより複雑な式で表すとどうなるでしょうか。`dim = 1`という部分の数値を変えてどうなるか見ていきましょう。

In [ ]:
import sympy as sym
from sympy.plotting import plot
dim = 7
x, y = sym.symbols("x y")

coefficients = np.polyfit(obs_x, obs_y, dim)
expr = 0
for index, coefficient in enumerate(coefficients):
    expr += coefficient * x ** (len(coefficients) - index - 1)
display(sym.Eq(y, expr))

x_latent = np.linspace(0, 10, 100)
fitted_curve = np.poly1d(np.polyfit(obs_x, obs_y, dim))(x_latent)
plt.scatter(obs_x, obs_y, color="black", linewidth=0)
plt.plot(x_latent, fitted_curve, c="red", label="fitted")

このように、過学習してしまうと、与えられたデータに適合しようとするあまり、他のデータに対する予測や当てはまりが上手くいかなくなってしまいます。

例えばx=10の時の値を予測してみると、過学習したモデルの場合おかしな値になってしまっているこいとが分かります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_overfit_2.png?raw=true" alt="GP_overfit_2" height="250px">

今回の例では次数を上げていった形になりますが、説明変数の数が多くなったりと、複雑なモデルになればなるほど過学習しやすい傾向にあります。

特に用意できたサンプル数(個体数)よりも説明変数の方が多い場合、過学習が起きやすく、計算結果がおかしな値になることが多くなります。

ゲノムデータを扱った解析では、SNPマーカーの数などがサンプル数に比べ非常に多いという状況が普通なため(p >> n問題と呼ばれる)、どの様にこの過学習に対応するか、が重要なポイントの１つになります。

# 3. 問題点を改善するために

さて、ここまで触れてきた様に、問題点はおおまかに以下の２点挙げられます。

* 他集団や新しいデータに対する予測精度が確認できない
* 過学習が起きてしまっている

ここからはこれらの問題を解決する方法を紹介していきます。

## 3-1. 学習データとテストデータ

まずは新しいデータに対する予測精度をどのように確認するか、です。

これは全てのデータを使って予測モデルを構築するのではなくて、

一部のデータを予測精度の確認用のデータとしてとっておき、残りのデータで予測モデルを構築する。という方法を取ります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_train_test.png?raw=true" alt="GP_train_test" height="400px">

この様に、モデルの構築に関わっていないデータをとっておくことで、構築した予測モデルが未知のデータに対してどのくらいの予測精度になるか、確認することが出来ます。

モデルを構築するためのデータを**トレーニングデータ**、精度を確認するためのデータを**テストデータ**と呼びます。


### 3-1-1. データの分け方

ではどのようにトレーニングデータとテストデータを分けるのかというと、

サンプル数100~10,000程度のデータセットでは、トレーニングデータ : テストデータを「70% : 30%」や、「75% : 25%」、「80% : 20%」の比率で分割するのが一般的です。サンプル数100,000では「90% : 10%」、1,000,000では「99% : 1%」といった比率が一般的です。

テストデータは確認用なので、全体のデータサイズが大きければその分割合は小さくて良いとする場合が多いです。

データの選び方は基本的に乱数等に従ってランダムに20%選ぶ、という形になりますが、データの分布には気を付けておく必要があります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_train_test2.png?raw=true" alt="GP_train_test2" height="400px">

形質値の分布が揃うようにデータを分割することもPythonのライブラリを使うと簡単に行うことが出来ます。


In [ ]:
from sklearn.model_selection import train_test_split

# 80%をトレーニングデータ、20%テストデータとして分ける
genotype = pd.read_csv("GP_sample_genotype.csv")
phenotype = pd.read_csv("GP_sample_phenotype.csv")
X_train, X_test, y_train, y_test = train_test_split(genotype.iloc[:, 2:].T, phenotype, test_size=0.2, random_state=100)
y_test

In [ ]:
# トレーニング、テストデータの分布確認
plt.hist(y_train.GN_mean, label="Training")
plt.hist(y_test.GN_mean, label="Test")
plt.legend()

## 3-1-2. (補足)乱数について

乱数とは０から９までの数字が不規則かつ等確率に現れるように配列されたものです。 学習の過程では様々な点で乱数が使用されています。 例えば、今紹介したトレーニングデータとテストデータにデータを8:2に分割する際に、乱数に基づいてデータが分割されています。 また、モデルの学習においてもパラメータの調整の変遷や初期値などに乱数が使用されていることも多いです。

下記のコードを動かしてみると、毎回トレーニングデータとテストデータの分けられ方が異なることが分かります。乱数は文字通りランダムに生成されるので、コードを動かすたびに異なる値が生成されています。

In [ ]:
# 乱数の確認
from sklearn.model_selection import train_test_split
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
print(x_train, x_test)
print(y_train, y_test)

この乱数は固定することが出来ます。 乱数を利用している関数にはseedやrandom_seed, random_stateという引数が用意されており、この値を決めておくことで同じ乱数のもと、関数を実行できます。 つまり、同じテストデータとトレーニングデータの分け方を再現可能になります。

In [ ]:
# 乱数を固定
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
print(x_train, x_test)
print(y_train, y_test)

サイエンスにおいて結果の「再現性」というものは非常に重要です。再現性というのは、

「XXという実験をYYという条件の下で行うと、ZZという結果が得られた」という実験結果が、何度繰り返しても、異なる実験者が行っても再現できるという事を示します。

一昔前にSTAP細胞に関する事件が起きましたが、あの件は他の研究者によってSTAP細胞の再現ができなかったため、虚偽ではないかとなったわけですね。

これは実験での話だけでなく、今皆さんがやっている計算やモデルの学習においても同じです。科学では、この計算も同じように「再現性」を取る必要があります。 そのため、通常は乱数を固定してデータの分け方や学習を行い、乱数を決めるrandom_seedなどの値はしっかり記録しておいて、どの様な乱数のもとデータを生成し学習していったのかを確実に再現できる様にしておく必要があります。

## 3-1-3. (補足)交差検証

(今回の実習では行いませんが)より汎用的なモデルを構築するためのデータ分割方法として交差検証(Cross-validation)と呼ばれるものがあります。

やはりテストデータが偏っていたり、偶然選択したテストデータを上手く予測出来てしまった、ということが発生します。

そこでその可能性を少しでも下げるために交差検証が用いられます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_CV.png?raw=true" alt="GP_CV" height="300px">

## 3-1-4. (参考)一般的な機械学習モデル構築の流れ

この様なテストデータの分割や交差検証というのは機械学習モデルを構築する際に一般的に用いられる手法です。

機械学習モデルというのは、データの特徴を学習して、何らかの入力データに対して求めたい出力結果を導き出す仕組みです。

例えば、Google翻訳であれば「日本語の文章」を入力すると「英語に翻訳された結果」が出力として予測されるモデルになります。

そのため、今回の「遺伝子型」を入力すると「形質値」が出力として予測されるモデルも、機械学習モデルと言えるでしょう。

今回の場合はデータから「遺伝子型と形質値の関係性」を学習した機械学習モデル、ということになります。

Genomic Predictionに限らず、機械学習モデルの構築は以下の様な流れで進みます。

1. 関心のあるデータ(入力+出力)の準備
2. トレーニングデータ、テストデータに分割
3. 手法を選択し、トレーニングデータからモデルを構築(入力から出力を導き出すモデル)
4. テストデータで構築したモデルの性能を確認(場合によっては交差検証を行う)
5. 良いモデルが出来れば予測に使用する

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/supervised_learning_process.png?raw=true" alt="supervised_learning_process" height="60px">

また、使用するモデルによってはパラメータの調整等も行う必要があるため、トレーニングデータ・テストデータだけでなくバリデーションデータと呼ばれるデータに更に分割する場合もあります。(今回は扱いません。)



## 3-1-5. 重回帰モデルの精度をテストデータで確認してみる

それでは前回全てのデータを使って回帰モデルを構築していたものを、トレーニング・テストデータに分割してやり直してみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_rice_process.png?raw=true" alt="GP_rice_process" height="350px">

まずはデータを80%と20%に分割します。

80%をトレーニングデータとして、モデル構築に使用し、残りの20%をモデルの精度を確認するためのテストデータにします。

In [ ]:
# 80%をトレーニングデータ、20%テストデータとして分ける
genotype = pd.read_csv("GP_sample_genotype.csv")
phenotype = pd.read_csv("GP_sample_phenotype.csv")
X_train, X_test, y_train, y_test = train_test_split(genotype.iloc[:, 2:].T, phenotype, test_size=0.2, random_state=100)
print("トレーニングデータ")
display(X_train)
display(y_train)
print("テストデータ")
display(X_test)
display(y_test)

続いて、トレーニングデータを使用して回帰モデルを構築します。

In [ ]:
# トレーニングデータからモデル構築
from sklearn.linear_model import LinearRegression

# X_train, y_trainから回帰モデルを作る
model = LinearRegression()
model = model.fit(X_train, y_train.GN_mean)
equation = "y = "
for i, j in enumerate(model.coef_):
    equation += "SNP{}×{} + ".format(str(i+1), str(j))
equation += str(model.intercept_)
equation

まずはこの回帰モデルが**トレーニングデータ**の遺伝子型と形質値の関係を説明出来ているか確認してみます。

In [ ]:
# トレーニングデータに対しての当てはまりの良さを確認
y_preds = model.predict(X_train)
plt.scatter(y_train.GN_mean, y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(y_train.GN_mean, y_preds)[0, 1]))

トレーニングデータから回帰モデルを作ったので当然といえば当然ですが、上手く説明は出来て良そうです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_training_acc.png?raw=true" alt="GP_training_acc" height="350px">

それでは次に、未知のデータ(新たなデータ)に対しても同じように良い予測性能を示すのか、**テストデータ**を用いて精度を見てみます。

In [ ]:
# テストデータで精度確認
y_test_preds = model.predict(X_test)          # X_test(テストデータの遺伝子型から形質値を予測)
plt.scatter(y_test.GN_mean, y_test_preds)   # 上で計算した予測値とy_test(実測値)を比較してみる
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(y_test.GN_mean, y_test_preds)[0, 1]))

未知のデータ(テストデータ)に対しては全く予測が上手くいっていないことが分かります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_test_acc.png?raw=true" alt="GP_test_acc" height="350px">

これは回帰式がトレーニングデータに過剰に適合してしまっている(過学習)ため、この様な結果となりました。

よって次は過学習を避けるにはどうすれば良いか、ということを考える必要があります。

## 3-2. 過学習を防ぐ手法を選択

過学習を避けるための方法の1つとして、**説明変数を減らす**というやり方があります。

前半部分で、説明変数の数がたくさんある複雑な回帰式になるほど過学習しやすくなるという話がありました。

そのため、本当に必要な限られた説明変数のみでモデルを構築することが過学習を避ける１つのポイントになります。

今回の場合も、全ゲノム領域の遺伝子型が形質に影響を与えているとは現実的には考えられず、原因遺伝子が位置する一部の領域の遺伝子型が形質に影響を与えている可能性の方が高そうなのはイメージできると思います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_not_all_gene.png?raw=true" alt="GP_not_all_gene" height="350px">

説明変数を絞って(**変数選択**と呼ばれたりします)モデルを構築する方法としては、大まかに以下の様な方法があります。

* ドメイン知識(生物学の知見等)等から、モデル構築に供与する変数を自分で選択する
* モデル構築する際に変数選択も同時に行ってくれる手法を使う

### 3-2-1 モデル構築に供与する変数を自分で選択する

例えば、遺伝子型から籾数を予測するモデルを作りたいとき、

「3番染色体に籾数に影響を与えるXXXという遺伝子が存在している」という情報がすでに分かっていれば、その領域付近のSNP遺伝子型は重要な変数になります。

他にも、遺伝子レベルでは無くても「7番染色体の20mbp付近を別の系統に入れ替えると籾数が飛躍的に伸びることが知られている」という報告がされていれば、その領域の遺伝子型も重要だと考えられますね。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_select.png?raw=true" alt="GP_select" height="350px">

このように既知の情報や知識を使って変数の選択を行う、というのが最初の方法になります。

加えて、GWASを行った結果から、重要そうな領域を検出し、その領域のみを使って予測モデルを構築する、というのもありですね。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_select2.png?raw=true" alt="GP_select2" height="240px">

つまり、すべての変数(今回だと全てのSNPの遺伝子型)を何も考えずに突っ込むのではなく、生物学的な知見や別の解析結果などを踏まえて、重要そうなものだけを使用してシンプルな予測モデルの構築を試みることで、過学習を避けようというやり方になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_select3.png?raw=true" alt="GP_select3" height="350px">

モデル植物や重要作物では遺伝子の機能がどんどん解明されているため、利用できる情報が多く、有効なアプローチの1つです。着目している遺伝子がどのくらいの効果を持っているのか調べたいときにも有効です。

しかし、そもそも形質に影響を与えている原因遺伝子等の要因が全て分かっているケースは殆どなく、重要な遺伝子を取りこぼす可能性があり、必要な情報が足りずに予測精度が低くなってしまう可能性も高いです。

また、研究の進んでいないマイナーな植物の場合、基準にする知見が不足していることが問題になってきます。

### 3-2-2. 変数選択も同時に行ってくれる手法を使う

上で紹介したドメイン知識に基づくやり方というのは、やはり必要な説明変数を全て拾いきれない可能性があります。

そこで、モデル構築の際に、変数選択も同時に行ってくれる手法を利用する、というのがもう1つのやり方になります。

例えば、前回理想的な遺伝子型や理想的な交配組み合わせを考える遊びをしましたが、その時のモデルはそのような手法を使って構築しています。


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from genomic_prediction import split_dataset, make_genomic_prediction_model, check_equation, predict_phenotype, check_accuracy
# トレーニングデータとテストデータに分割
test_genotype, test_phenotype, train_genotype, train_phenotype = split_dataset(genotype, phenotype, "GN_mean", test=0.2)
# トレーニングデータから予測モデルを構築
GN_prediction_model = make_genomic_prediction_model(train_genotype, train_phenotype, "GN_mean")
print("\nPrediction model is ...")
print(check_equation("GN_mean", GN_prediction_model), "\n")

plt.figure(figsize=(12,3))
plt.plot(GN_prediction_model.coef_)
plt.xlabel("SNP")
plt.ylabel("coefficient (SNP effect)")
plt.show()

全てのSNP情報が使われているわけではなく、一部のSNPのみを用いて、式が構築されているのが分かると思います。

この式はモデルを作る過程で不要なSNPの情報をどんどん削ぎ落していくような手法を使って作られています。

このモデルを用いてテストデータに対して予測精度を確認してみると…

In [ ]:
# テストデータでの精度確認
y_preds = GN_prediction_model.predict(X_test)
plt.scatter(y_test.GN_mean, y_preds)
plt.title("Test data")
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(y_test.GN_mean, y_preds)[0, 1]))

この様に未知のデータに対する予測精度も高いことが確認できます。

今回は、モデル構築の際に変数選択も同時に行ってくれる最も簡単な手法の代表例として`Lasso回帰`という手法を紹介します。

### 3-2-3 Lasso回帰

(参考程度にどういう手法か書いておきますが、興味があればでokです。)

Lasso回帰というのは回帰分析手法の１つになります。

重回帰モデルの学習では、以下の様な式で、

$$ y = \beta_{1} x_{1} + \beta_{2} x_{2} + ...  + \beta_{k} x_{k} + e $$

残差の二乗の合計値、**目的関数**である**残差平方和 (residual sum of squares)** が最も小さくなる係数 $\beta$ と誤差 $e$ の直線を求めていました。

$$ 残差平方和: \sum_{i=1}^{N} (\hat{y}_{i} - y_i)^2 $$

<img src="https://lh3.googleusercontent.com/pw/ACtC-3eIFmh8PDRx64eFArwdgxO2CGt3PEi272ny1dyqAMue0un_yL_GMgZ0CsyvBnX4lEC9BfOEdfTNGsiEG-R4xZDPM9zMHwHcINcnQFxcdTmSgsF7LotLsBpwzs0S49fZtN1fQrbHY7JrB9m2kwuDGb9r=w815-h560-no?authuser=0" alt="least squares" height="180px">

Lassoでは、この目的関数に正則化項を入れて過学習を抑制する手法になります。

正則化では、この残差平方和に正則化項(罰則項)というものを加え、この値を最小化するような係数 $\beta$ と誤差 $e$ を求めます。

$$ \sum_{i=1}^{N} (\hat{y}_{i} - y_i)^2 + \alpha\sum_{j=1}^{k} |\beta_j|$$

正則化項を加えることで、全体的に偏回帰係数 $\beta$ の重みの影響を制限しようとする (係数の絶対値を小さくする) 形になります。

今回だと$\beta$の値は各SNPの効果を表していますが、あんまり大きな効果をSNPに与えるとペナルティが増えてしまう感じです。

結果的に、出来るだけ最小限の$\beta$だけで残差平方和が小さくなるような式を求めることが出来る、ということになります。

また、$\alpha$の値を大きくすることでペナルティを強くすることができ、より少ないSNPを使った状態で残差平方和が最初になる様な式を求めることが出来ます。

($\alpha=0$の時、普通の重回帰分析と同じになる。)

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_alpha.png?raw=true" alt="GP_alpha" height="350px">

実際にやってみましょう。



In [ ]:
# Lasso回帰
from sklearn.linear_model import Lasso
model = Lasso(alpha=0.01) #このalphaの値を大きくすると使用する変数を更に減らせる。※alpha=0は数値計算上使わない方が良い。
model = model.fit(X_train, y_train.GN_mean)
equation = "y = "
k = 0
for i, j in enumerate(model.coef_):
    if j == 0:
        pass
    else:
        k += 1
        equation += "SNP{}×{} + ".format(str(i+1), str(j))
equation += str(model.intercept_)

print("説明変数に使用したSNPの数:", k)
print(equation)

plt.figure(figsize=(12,3))
plt.plot(model.coef_)
plt.xlabel("SNP")
plt.ylabel("coefficient (SNP effect)")
plt.show()

In [ ]:
# トレーニングデータに対しての当てはまりの良さを確認
y_preds = model.predict(X_train)
plt.scatter(y_train.GN_mean, y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(y_train.GN_mean, y_preds)[0, 1]))

In [ ]:
# テストデータで精度確認
y_test_preds = model.predict(X_test)
plt.scatter(y_test.GN_mean, y_test_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(y_test.GN_mean, y_test_preds)[0, 1]))

単純な線形回帰の時よりも、テストデータ(未知のデータ)に対する予測精度が上昇したかと思います。

### 3-2-4. その他の手法について

今回はLasso回帰を紹介しましたが、回帰分析の手法には他にもRidge回帰やElasticNetなど、様々な手法が存在しています。

また回帰ではなく決定木と呼ばれる手法を元にしたRandomForestやGradientBoostingDecisionTree(GBDT)等、予測モデルを構築する手法は無数にあります。

それぞれの手法に特徴があり、どの様な要因を考慮するかで採用する手法が変わります。

(遺伝子間の相互作用やより高次の相互作用を考慮するなら、線形モデルではなく、別の手法を採用する必要がある、等)

各手法について説明していると統計学の話になってしまうので、興味がある人は以下の書籍を見てみてください。

統計的学習とは何か等データサイエンスについての考え方も書いてあります。

日本語に翻訳された本は高いですが、英語版であればpdfが無料で配布されています。

* Rによる統計的学習入門 (朝倉書店) G. James et al.,
* (英語版は無料) [An Introduction to Statistical Learning with Applications in R](https://hastie.su.domains/ISLR2/ISLRv2_website.pdf)

* 統計的学習の基礎 (共立出版) T. Hastie et al.,
* (英語版は無料) [The Elements of Statistical Learning](https://hastie.su.domains/Papers/ESLII.pdf)

※入門とか基礎とか書いてますが確率論や線形代数の基礎知識は必要です。

# まとめ

　今回、Genomic Predictionを単純な回帰モデルで構築する場合の問題点・改善方法を学びました。

- 説明変数が多すぎると過学習を起こす。
- 過学習を避けるには、変数選択をしてよりシンプルなモデルにするのが１つの解決策
  - 変数選択には、ドメイン知識を元にした選択手法や、モデル構築の際に選択する手法などがある。

というわけで、これまで集団のDNAの情報を使ってQTLを探す方法や、遺伝子型の情報から形質値の予測を試みることが出来る、

ということを学びました。

次回からはDNAではなくRNAを対象としたシーケンス技術を使ってどのような事が出来るのか学んでいきます。

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>